In [ ]:
import re

def clean(row1):
    modified_string = re.sub(r"\<[^<>]*\>", "", row1)    #remove tags
    result = re.sub(r"\[[^<>]*\]", "", modified_string)  #remove callbacks
    return(result)

In [ ]:
import re
import json
from sudachipy import tokenizer, dictionary

SENTENCE_SEP = '\t'
ENTITY_SEP = '-'


def tokenize(sentence, mode):
    """ To perform tokenization on the train sentences and the values """
    tokenizer_obj = dictionary.Dictionary().create()
    if mode == "A":
        split_mode = tokenizer.Tokenizer.SplitMode.A
    elif mode == "B":
        split_mode = tokenizer.Tokenizer.SplitMode.B
    elif mode == "C":
        split_mode = tokenizer.Tokenizer.SplitMode.C
    else:
        raise ValueError("Invalid Mode: only A, B or C is allowed")

    return "".join([m.surface() for m in tokenizer_obj.tokenize(sentence, split_mode)])

In [ ]:
#f = open("scr3_new.txt", "w")


#_ = str(row["Labeled Cleaned Transcription"])
_ = 'はい、かしこまりました<PERSON>渋谷</PERSON>様ですね.はい、 [filler/] お客様情報確認ができました.'
flag = 0
q = ""
p =''
temp = clean(_)
for i in range(len(_)): 
    tag=''
    entity=''
    if _[i]=='<' and _[i+1] != "/":
        flag = 1
        i=i+1
        while _[i] != ">":
            tag = tag + _[i]
            i=i+1
        i=i+1
        while _[i] != "<":
            entity = entity + _[i]
            i=i+1
        #p = p + entity + "\n"
        if q == "":
            q = q + entity + " - " + tag
        else:
            q = q + "\t" + entity + " - " + tag
if flag:
    p = temp + "\t" + q
    #f.write(p + "\n")
    #if tag!="" and entity != "":
    #    print(tag,entity)
    # 大阪ガスお客様センター - ORG,小池 - PERSON
else:
    p = temp
    #f.write(p + "\n")
print(p)
#f.close()

In [ ]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding
import json
# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import pickle
import warnings
import numpy as np
import pickle
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
import warnings
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings("ignore")

nlp = spacy.load('ja_core_news_lg')

with open('new_data_sudachy.json', mode='r', encoding='utf8') as train_file:
    train_json = json.load(train_file)
train_data = train_json["train_data"]

with open('krypton_eval_sudachy_new.json', mode='r', encoding='utf8') as train_file:
    train_json = json.load(train_file)
test_data = train_json["train_data"]

def spacy_retrain(train_data_file, new_model_location):
    with open(train_data_file, mode='r', encoding='utf8') as train_file:
        train_json = json.load(train_file)
    train_data = train_json["train_data"]

    # For reproducing same results during multiple run
    s = 42
    np.random.seed(s)
    spacy.util.fix_random_seed(s)
    
    #optimizer = Adam(
    #learning_rate=0.001
    #)
    optimizer = nlp.entity.create_optimizer()
    
    ner = nlp.get_pipe('ner')
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    print(f'[OtherPipes] = {other_pipes} will be disabled')

    model = nlp
    X = []
    y = []
    for text, annotations in train_data:
        X.append(text)
        y.append(annotations)

    n_iter = 5
    loss = []
    with nlp.disable_pipes(*other_pipes):  # only train ner
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.resume_training()
        for i in range(n_iter):
            losses = {}
            nlp.update(X, y, sgd=optimizer, drop=0.25, losses=losses)
            # nlp.entity.update(d, g)
            print("Losses", losses)
            loss.append(losses['ner'])
            if losses['ner'] <= min(loss):
                flag = i
                nlp.to_disk(new_model_location)
    return loss, flag
      

train_file = r'full_filler_scr.json'
new_model_loc = r'./full_fuller_try'
loss,f = spacy_retrain(train_file, new_model_loc)

In [ ]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        
        scorer.score(pred_value, gold)
        
        
    return scorer.scores

model = loadNERModel('./full_scr1_filler/')
print(evaluate(model, test_data))